<a id='report-interactive'></a>
<p style="font-weight:bold;"> <span style="font-size: 36px"> Interactive Reports</span> </p>

# Set up data and configuration

Choose to run the Reports notebook either with the set of Systemorph data in memory or with the data present in the Database: 
- #!eval-notebook "../Database/Configure" : connects to a physical Database
- #!eval-notebook "../Import/CloseImportTemplate" : uses the in-memory set up

We use here the in-memory set up.

In [0]:
#!import "../Import/CloseImportTemplate"

In [0]:
Workspace.InitializeFrom(DataSource);
ifrs17Interactive.Reset(Workspace)

# Best Estimate

Present values of the [best-estimate](https://portal.systemorph.cloud/project/ifrs17/env/v1.1.0/Report/ReportScopes#best-estimate) future cash flows are shown here in an Analysis of Change report.

The granularity of the reported figures can be modified by changing the Column Slices options.
For example one can add <code>"GroupOfContract"</code> to separate the contributions of the individual Group of Contracts.

Aggregated values are displayed when the data has a finer granularity than the one selected by the report slice options.

In [0]:
ifrs17Interactive.GetFormsEntity<PvReport>()

In [0]:
ifrs17Interactive.GetReport<PvReport>()